In [78]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords
from langdetect import detect
from sklearn.utils import shuffle
#from nltk.stem.porter import PorterStemmer

Data Processing

In [79]:
#importing the datasets
data1_fake = pd.read_csv('./Data_RealFake/Dataset1/Fake1.csv')
data1_true = pd.read_csv('./Data_RealFake/Dataset1/True1.csv')

dtset1_fake = pd.DataFrame(data1_fake)
dtset1_true = pd.DataFrame(data1_true)

In [80]:
dtset1_fake['label']=0
dtset1_true['label']=1

pd.DataFrame.head(dtset1_fake)
pd.DataFrame.head(dtset1_true)

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [81]:
print(dtset1_fake.shape)
print(dtset1_true.shape)

(23481, 5)
(21417, 5)


In [82]:
dataset1 = pd.concat([dtset1_fake, dtset1_true])
pd.DataFrame.head(dataset1)

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [83]:
dataset1.label.value_counts()

0    23481
1    21417
Name: label, dtype: int64

In [84]:
dataset1 = shuffle(dataset1)
dataset1[:10]

,title,text,subject,date,label
8579,Clinton leads Trump by 6 points after Democrat...,NEW YORK (Reuters) - Democratic presidential ...,politicsNews,"July 29, 2016",1
11908,HYPOCRISY ON STEROIDS: Check Out HATEFUL Trump...,"Lady Gaga, who protested in front of Trump Tow...",politics,"Jan 14, 2017",0
20290,"London seeks ""deep security partnership"" with ...",LONDON (Reuters) - Britain wants to have close...,worldnews,"September 11, 2017",1
4964,Veteran U.S. diplomat Patterson out of running...,WASHINGTON (Reuters) - Veteran U.S. diplomat A...,politicsNews,"March 14, 2017",1
1299,Dan Rather: ‘We Haven’t Had A President This ...,Respected journalist Dan Rather appeared on Th...,News,"June 2, 2017",0
9526,Betting on a Trump win or a North Korea H-bomb...,NEW YORK/WELLINGTON (Reuters) - Erik Duhaime i...,politicsNews,"May 16, 2016",1
19014,LOUDMOUTH CELEBRITIES Forced To EAT CROW After...,"On March 6, newly-minted Secretary of Housing ...",left-news,"Mar 8, 2017",0
17948,Any Brexit deal will come at end of two-year n...,LONDON (Reuters) - Prime Minister Theresa May ...,worldnews,"October 9, 2017",1
9738,Congressman wants Homeland Security to address...,(Reuters) - A U.S. congressman called on the D...,politicsNews,"April 28, 2016",1
869,Apparently Donald Trump Has An Imaginary Xeno...,Now that we ve known Donald Trump well for a w...,News,"July 12, 2017",0
